In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#!pip install google-cloud-bigquery

In [ ]:
#!pip install langchain

In [ ]:
#!pip install langchain-experimental


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
#!pip install openai


In [ ]:
#!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os
from google.cloud import bigquery
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
import concurrent.futures
import time
from neo4j import GraphDatabase
import json
from openai import OpenAI
from tqdm import tqdm

In [ ]:
# Define path to service account key in Google Drive
key_path = "/content/drive/My Drive/Colab Notebooks/zinc-direction-451800-q6-9dee276728e4.json"

# Set the environment variable for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
# Initialize BigQuery client
client = bigquery.Client()
query = """
SELECT *
FROM `zinc-direction-451800-q6.mimic_dataset.llm-train`
LIMIT 100
OFFSET 300;
"""
print("Fetching data from BigQuery...")
df = client.query(query).to_dataframe()
print(f"✅ Fetched {len(df)} records from BigQuery")


Fetching data from BigQuery...
✅ Fetched 100 records from BigQuery


In [ ]:
# For the first row with all columns displayed
print("FIRST ROW (FULL):")
with pd.option_context('display.max_columns', None, 'display.width', 1000):
    print(df.iloc[0])

# For the last row with all columns displayed
print("\nLAST ROW (FULL):")
with pd.option_context('display.max_columns', None, 'display.width', 1000):
    print(df.iloc[-1])

FIRST ROW (FULL):
output     you require an opg radiograph, impressions of ...
input      hithanks for your reply. i have now got a orth...
context                                                     
Name: 0, dtype: object

LAST ROW (FULL):
output     lower back pain is not related to masturbation...
input      Hello sir, I am 18 yr old and i have been mast...
context                                                     
Name: 99, dtype: object


In [ ]:
# Load the config from the JSON file
config_path = "/content/drive/My Drive/Colab Notebooks/Config2.json"
with open(config_path, 'r') as f:
    config = json.load(f)

# Access the Neo4j details
neo4j_config = config["neo4j"]
URI = neo4j_config["URI"]
USERNAME = neo4j_config["USERNAME"]
PASSWORD = neo4j_config["PASSWORD"]

# Access the OpenAI API key
openai_config = config["openai"]
os.environ["OPENAI_API_KEY"] = openai_config["API_KEY"]

# Initialize timer
start_time = time.time()

In [ ]:
#!pip install json-repair

In [ ]:
#!pip install langchain langchain-openai langchain-community langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.47
    Uninstalling langchain-core-0.3.47:
      Successfully uninstalled langchain-core-0.3.47


In [ ]:
#!pip install neo4j tqdm matplotlib networkx tabulate openai google-cloud-bigquery

#Transform and Load

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=openai_config["API_KEY"])

# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [ ]:
# Process a row of data to extract medical knowledge graph
def extract_medical_graph(row):
    prompt = f"""
You are a medical knowledge graph extraction expert. Extract entities and relationships from this medical consultation:

CONTEXT: {row.get('context', '')}
PATIENT QUESTION: {row.get('input', '')}
DOCTOR RESPONSE: {row.get('output', '')}

Extract the following types of nodes:
1. Patient - Extract patient information (age, gender, existing conditions)
2. Symptom - Identify all symptoms mentioned
3. Medication - Extract all medications mentioned
4. MedicalCondition - Identify medical conditions
5. Treatment - Extract treatments or recommendations
6. MedicalDevice - Extract medical devices (like Copper T)
7. Doctor - Extract doctor information when present

Extract these relationship types:
1. (Patient)-[HAS_SYMPTOM]->(Symptom)
2. (Patient)-[HAS_CONDITION]->(MedicalCondition)
3. (Patient)-[USES]->(Medication)
4. (Patient)-[USES_DEVICE]->(MedicalDevice)
5. (Doctor)-[RECOMMENDS]->(Treatment)
6. (Treatment)-[TREATS]->(MedicalCondition)
7. (Medication)-[CAUSES]->(Symptom) - For side effects
8. (MedicalCondition)-[PRESENTS_WITH]->(Symptom)

For each node and relationship, extract relevant properties.

Return ONLY a JSON object with this exact structure:
{{
  "nodes": [
    {{
      "id": "unique_id",
      "label": "NodeType",
      "properties": {{
        "property1": "value1",
        "property2": "value2"
      }}
    }}
  ],
  "relationships": [
    {{
      "source_id": "node_id_1",
      "target_id": "node_id_2",
      "type": "RELATIONSHIP_TYPE",
      "properties": {{
        "property1": "value1"
      }}
    }}
  ]
}}

IMPORTANT: Use consistent IDs for the same entities across the graph. Make IDs descriptive (e.g., "Patient_1", "Symptom_headache").
"""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=2000
        )

        # Extract JSON from response
        content = response.choices[0].message.content
        # Find JSON part
        json_start = content.find('{')
        json_end = content.rfind('}') + 1
        if json_start == -1 or json_end == 0:
            print("No JSON found in response")
            print("Response content:", content)
            return {"nodes": [], "relationships": []}

        json_str = content[json_start:json_end]

        data = json.loads(json_str)
        return data
    except Exception as e:
        print(f"Error in extraction: {e}")
        print(f"Response content: {content if 'content' in locals() else 'No response received'}")
        return {"nodes": [], "relationships": []}

In [ ]:
# Function to create nodes in Neo4j
def create_nodes(tx, nodes):
    for node in nodes:
        node_id = node.get("id")
        label = node.get("label")
        properties = node.get("properties", {})

        # Add ID to properties
        all_props = {"id": node_id, **properties}

        # Convert properties to Cypher parameters
        props_list = []
        params = {}

        for k, v in all_props.items():
            param_name = f"prop_{k}"
            props_list.append(f"{k}: ${param_name}")
            params[param_name] = v

        props_str = ", ".join(props_list)

        # Create node
        query = f"MERGE (n:{label} {{{props_str}}}) RETURN n"

        try:
            tx.run(query, **params)
        except Exception as e:
            print(f"Error creating node: {e}")
            print(f"Query: {query}")
            print(f"Params: {params}")
            raise

In [ ]:
# Function to create relationships in Neo4j
def create_relationships(tx, relationships):
    for rel in relationships:
        source_id = rel.get("source_id")
        target_id = rel.get("target_id")
        rel_type = rel.get("type")
        properties = rel.get("properties", {})

        # Convert properties to Cypher parameters
        props_list = []
        params = {"source": source_id, "target": target_id}

        for k, v in properties.items():
            param_name = f"prop_{k}"
            props_list.append(f"{k}: ${param_name}")
            params[param_name] = v

        props_str = ", ".join(props_list)
        if props_str:
            props_str = " {" + props_str + "}"

        # Create relationship
        query = f"""
        MATCH (a), (b)
        WHERE a.id = $source AND b.id = $target
        MERGE (a)-[r:{rel_type}{props_str}]->(b)
        RETURN r
        """

        try:
            tx.run(query, **params)
        except Exception as e:
            print(f"Error creating relationship: {e}")
            print(f"Query: {query}")
            print(f"Params: {params}")
            raise

In [ ]:
# Process a single row and add to Neo4j
def process_row_to_neo4j(row, driver):
    # Extract graph data
    data = extract_medical_graph(row)

    # Log extraction results
    print(f"Extracted {len(data['nodes'])} nodes and {len(data['relationships'])} relationships")

    # Check if we have data to add
    if not data['nodes'] and not data['relationships']:
        return False

    # Add to Neo4j
    with driver.session() as session:
        try:
            # Add nodes first
            session.execute_write(create_nodes, data['nodes'])

            # Then add relationships
            session.execute_write(create_relationships, data['relationships'])

            return True
        except Exception as e:
            print(f"Error adding to Neo4j: {e}")
            return False


In [ ]:
# Process entire dataframe
def process_dataframe(df, driver, batch_size=5):
    successful = 0
    total = len(df)

    print(f"Processing {total} records in batches of {batch_size}")

    for i in tqdm(range(0, total, batch_size)):
        batch = df.iloc[i:min(i+batch_size, total)]

        for _, row in batch.iterrows():
            success = process_row_to_neo4j(row.to_dict(), driver)
            if success:
                successful += 1

            # Prevent rate limiting
            time.sleep(1)

    print(f"Successfully processed {successful}/{total} records")

    return successful

In [ ]:
# Run the extraction
start_time = time.time()
successful = process_dataframe(df, driver, batch_size=5)
elapsed_time = time.time() - start_time
print(f"Extraction completed in {elapsed_time:.2f} seconds")

Processing 100 records in batches of 5


  0%|          | 0/20 [00:00<?, ?it/s]

Extracted 5 nodes and 6 relationships
Extracted 4 nodes and 4 relationships
Extracted 13 nodes and 12 relationships
Extracted 6 nodes and 6 relationships
Extracted 8 nodes and 7 relationships


  5%|▌         | 1/20 [00:39<12:38, 39.94s/it]

Extracted 7 nodes and 8 relationships
Extracted 5 nodes and 5 relationships
Extracted 5 nodes and 6 relationships
Extracted 7 nodes and 7 relationships
Extracted 7 nodes and 8 relationships


...

Extracted 7 nodes and 9 relationships
Extracted 6 nodes and 6 relationships
Extracted 7 nodes and 13 relationships
Extracted 6 nodes and 9 relationships
Extracted 7 nodes and 5 relationships


 95%|█████████▌| 19/20 [14:52<00:52, 52.21s/it]

Extracted 4 nodes and 4 relationships
Extracted 7 nodes and 6 relationships
Extracted 4 nodes and 4 relationships
Extracted 10 nodes and 9 relationships
Extracted 4 nodes and 4 relationships


100%|██████████| 20/20 [15:32<00:00, 46.63s/it]

Successfully processed 100/100 records
Extraction completed in 932.60 seconds


In [ ]:
# def delete_existing_data():
#     print("🧹 Deleting existing data in Neo4j...")
#     driver = GraphDatabase.driver(
#         URI,
#         auth=(USERNAME, PASSWORD)
#     )
#     with driver.session() as session:
#         session.run("MATCH (n) DETACH DELETE n")
#         print("✅ Existing data deleted successfully from Neo4j.")
# delete_existing_data()

🧹 Deleting existing data in Neo4j...
✅ Existing data deleted successfully from Neo4j.


In [ ]:
# Install the necessary package
!pip install yfiles_jupyter_graphs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
import concurrent.futures

In [ ]:
# !pip install yfiles-jupyter-graphs

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
import IPython.display   # Import as a module, not as a function

try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

def showGraph(cypher="MATCH p=(s:Symptom)--(c:MedicalCondition) RETURN p LIMIT 50"):
    # Create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri=URI,
        auth=(USERNAME, PASSWORD)
    )

    with driver.session() as session:
        result = session.run(cypher)
        graph_data = result.graph()

        # Create and configure widget
        widget = GraphWidget(graph=graph_data)

        # Try to set node labels based on properties that might exist
        def get_node_label(node):
            props = node.properties
            if 'name' in props and props['name']:
                return props['name']
            elif 'description' in props and props['description']:
                return props['description']
            elif 'id' in props and props['id']:
                return props['id']
            else:
                return node.labels[0]

        widget.node_label_function = get_node_label
        widget.height = 600

        # Use IPython.display.display() explicitly
        IPython.display.display(widget)
        return widget

# Example usage:
showGraph("MATCH p=(s:Symptom)--(c:MedicalCondition) RETURN p LIMIT 25")

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import IPython.display as ipd

def query_neo4j(cypher_query):
    driver = GraphDatabase.driver(
        uri=URI,
        auth=(USERNAME, PASSWORD)
    )

    # Execute query and get results
    with driver.session() as session:
        result = session.run(cypher_query)
        # Convert result to a list of dictionaries
        records = [dict(record) for record in result]

    driver.close()
    return records

# Get symptom-condition pairs (using your actual relationship PRESENTS_WITH)
symptom_condition_results = query_neo4j("""
    MATCH (c:MedicalCondition)-[r:PRESENTS_WITH]->(s:Symptom)
    RETURN
        COALESCE(s.name, s.description, s.id) as Symptom,
        COALESCE(c.name, c.description, c.id) as Condition
    LIMIT 50
""")

# Get medication side effects
medication_side_effects = query_neo4j("""
    MATCH (m:Medication)-[r:CAUSES]->(s:Symptom)
    RETURN
        COALESCE(m.name, m.description, m.id) as Medication,
        COALESCE(s.name, s.description, s.id) as SideEffect
    LIMIT 50
""")

# Get treatment effectiveness
treatment_results = query_neo4j("""
    MATCH (t:Treatment)-[r:TREATS]->(c:MedicalCondition)
    RETURN
        COALESCE(t.name, t.description, t.id) as Treatment,
        COALESCE(c.name, c.description, c.id) as Condition
    LIMIT 50
""")

# Display symptom-condition pairs as a table
print("Symptom-Condition Relationships:")
df_symptom_condition = pd.DataFrame(symptom_condition_results)
ipd.display(df_symptom_condition)  # Use the aliased display function

# Display medication side effects as a table
print("\nMedication Side Effects:")
df_side_effects = pd.DataFrame(medication_side_effects)
ipd.display(df_side_effects)  # Use the aliased display function

# Display treatment effectiveness as a table
print("\nTreatment-Condition Relationships:")
df_treatments = pd.DataFrame(treatment_results)
ipd.display(df_treatments)  # Use the aliased display function

Symptom-Condition Relationships:


,Symptom,Condition
0,trouble coughing until vomiting,Chronic Obstructive Pulmonary Disease
1,sore bronchial tubes,Chronic Obstructive Pulmonary Disease
2,No sense in left foot,Infection in ankle joint
3,irregular vaginal bleeding,hormonal imbalances
4,whiteheads,acne
5,pimples,rosacea
6,pimples,hypertrichosis
7,moderate dilatation of ureter and pelvicalicea...,VUJ calculus
8,Symptom_pain_jaw,sharp bony margin not smoothened after tooth r...
9,Symptom_numbness_lip_chin,injury to the inferior alveolar nerve



Medication Side Effects:


,Medication,SideEffect
0,Advair,heavy feeling in lungs
1,birth control pill,irregular vaginal bleeding
2,Medication_paracetamol,Fever
3,Medication_paracetamol,fever
4,Medication_paracetamol,Low grade fever
5,Medication_paracetamol,fever
6,antibiotics,Fever
7,antibiotics,fever
8,antibiotics,pain
9,antibiotics,Diarrhea



Treatment-Condition Relationships:


,Treatment,Condition
0,Pulmonary rehabilitation,Chronic Obstructive Pulmonary Disease
1,Visit the ER immediately,Gas gangrene
2,Treatment_surgical_removal,MedicalCondition_inflamed_sebaceous_cyst
3,take unwanted pill within 72 hours of unprotec...,unwanted pregnancy
4,assisted reproductive techniques,bilateral grade 2 vericocele
5,physical and clinical assessment of lumps,MedicalCondition_lymphnode_enlargement
6,physical and clinical assessment of lumps,MedicalCondition_lipoma
7,"X-ray, USG, CT study",MedicalCondition_lymphnode_enlargement
8,"X-ray, USG, CT study",MedicalCondition_lipoma
9,consult a physician and get the vitals checked,low blood sugar level
